In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sn
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from IPython.display import clear_output
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout, InputLayer, Activation, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

# adding path so as to import created .py file
import sys
sys.path.append('../') 

# importing a function to visualize live training result 
from plotlearning import PlotLearning       

2023-08-13 17:03:03.574060: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 17:03:03.594007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 17:03:03.737427: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 17:03:03.738386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 17:03:04.630423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

For this notebook we are gonna predict GOOGLE's stock price 

In [2]:
data=yf.download("GOOG")

[*********************100%***********************]  1 of 1 completed


In [3]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-08-19,2.490664,2.591785,2.390042,2.499133,2.499133,897427216
2004-08-20,2.515820,2.716817,2.503118,2.697639,2.697639,458857488
2004-08-23,2.758411,2.826406,2.716070,2.724787,2.724787,366857939
2004-08-24,2.770615,2.779581,2.579581,2.611960,2.611960,306396159
2004-08-25,2.614201,2.689918,2.587302,2.640104,2.640104,184645512
...,...,...,...,...,...,...
2023-08-07,129.509995,132.059998,129.429993,131.940002,131.940002,17621000
2023-08-08,130.979996,131.940002,130.130005,131.839996,131.839996,16836000
2023-08-09,132.190002,132.470001,129.505005,130.149994,130.149994,17745200


For this notebook we will predict Closing price of 'Google' stocks 

In [4]:
df=data['Close'].copy().reset_index()

In [5]:
df.head()

,Date,Close
0,2004-08-19,2.499133
1,2004-08-20,2.697639
2,2004-08-23,2.724787
3,2004-08-24,2.611960
4,2004-08-25,2.640104


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4778 entries, 0 to 4777
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4778 non-null   datetime64[ns]
 1   Close   4778 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 74.8 KB


Let's see how stock price went over past days

In [7]:
fig=go.Figure()
fig.add_scatter(x=df['Date'],y=df['Close'])
fig.update_layout(template="plotly_dark",
                  hovermode='x unified',
                  xaxis_title="Date",
                  yaxis_title="Close",
                  title="Google's Stock Price")


google's stock price is increasing, how ever a downfall can be seen after the end of year 2021<br>
and its increased after the year 2022 ends  

In [8]:
# Scaling our data as LSTM needs data to be normalizied
le = MinMaxScaler()

scaled_close_price = le.fit_transform(df.iloc[:,1:])

In [10]:
scaled_close_price

array([[5.54601395e-05],
       [1.39474256e-03],
       [1.57790515e-03],
       ...,
       [8.61292193e-01],
       [8.61697089e-01],
       [8.61427158e-01]])

In [11]:
scaled_close_price.shape

(4778, 1)

In [12]:
# defining how many past days model will look and will predict future
backcandles=30      # choosing 30 days 

We need to prepare our data so as <br>
model will train on past 30 days and will predict the future as y 

In [13]:
# reparing the data

X=np.array([scaled_close_price[i:i+backcandles] for i in range(len(scaled_close_price)-backcandles)])

In [14]:
# here is my target column
Y=scaled_close_price[backcandles:,]

In [15]:
X.shape,Y.shape

((4748, 30, 1), (4748, 1))

In [16]:
# we will split 90% of our data as training set, 5% will be validation set and rest will be test set
x_train = X[:int(X.shape[0]*.9)]
y_train = Y[:int(X.shape[0]*.9)]
x_val = X[int(X.shape[0]*.9):int(X.shape[0]*.95)]
y_val = Y[int(X.shape[0]*.9):int(X.shape[0]*.95)]
x_test = X[int(X.shape[0]*.95):]
y_test = Y[int(X.shape[0]*.95):]

In [17]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape

((4273, 30, 1), (4273, 1), (238, 30, 1), (238, 1), (237, 30, 1), (237, 1))

In [ ]:
kamal

In [18]:
# Already saved model for you, so you dont have to
model = keras.models.load_model("./saved_model")

In [26]:
# defifing LSTM model

model = Sequential()
model.add(LSTM(64,input_shape=(backcandles,x_train.shape[-1]),return_sequences=False))
# model.add(LSTM(32,return_sequences=False))
# model.add(LSTM(16,return_sequences=False))
model.add(Dropout(.1))
model.add(Dense(1,'linear'))

In [27]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='mse',metrics=['RootMeanSquaredError'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(x=x_train,y=y_train,
                    validation_data=(x_val,y_val),
                    epochs = 300, batch_size = 64,
                    verbose=0,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=30,verbose=0),PlotLearning()])

In [20]:
# predictions and errors
y_predicted=model.predict(x_test)
print("RMSE : ",np.mean(np.square(y_test-y_predicted)))

8/8 [==============================] - 1s 5ms/step
RMSE :  0.00045256278429566796


In [21]:
# the dates availabe in test set
test_dates = df['Date'][-x_test.shape[0]-1:-1]

In [22]:

fig=go.Figure()
fig.add_scatter(x=test_dates,y=le.inverse_transform(y_predicted).ravel(),
                marker=dict(color="#00ffff"),
                name="predictions")
fig.add_scatter(x=test_dates,y=le.inverse_transform(y_test).ravel(),
                marker=dict(color="#ff6200"),
                name="Actual")
fig.update_layout(template="plotly_dark",
                  hovermode='x unified',
                  xaxis_title="Date",
                  yaxis_title="Close",
                  title="A quick comparision")


In [ ]:
# model.save('./saved_model/')

In [24]:
# previously we are fedding models actual past value to predict the nextday value
# but what if we give model's predicte value as past value

# Creating a data frame with predited values


updated_x = x_val[-1:]

for i in range(len(x_test)):
    
    updated_x=np.append(updated_x,[model.predict(updated_x[:,-backcandles:],verbose=0)],axis=1)
    


In [25]:

fig=go.Figure()
fig.add_scatter(x=test_dates,y=le.inverse_transform(updated_x[0][backcandles:]).ravel(),
                marker=dict(color="#00ffff"),
                name="predictions")
fig.add_scatter(x=test_dates,y=le.inverse_transform(y_test).ravel(),
                marker=dict(color="#ff6200"),
                name="Actual")
fig.update_layout(template="plotly_dark",
                  hovermode='x unified',
                  xaxis_title="Date",
                  yaxis_title="Close",
                  title="predictions if past values are unknown")


For a longer run it shows price will decrease but in reaity it should be increasing

So our model perfoms good it we want to predict only tomorrow's Closing price, <br>
But if we want to forcast for a longer peiod of time like in this case we did for 238 days , then its bad <br>
upto 10 days it is giving a suitable result


In [9]:
# dumping saler

import joblib

joblib.dump(le, "scaler.pkl")


['scaler.pkl']